# Use GriddedModel3D object to manipulate Ndarray NAM model

First we will instantiate the GriddedModel3D object using the ndaray and header data created from the NAM Piece-Wise model.  Then we will see how to:
* get a depth slice by index 
* get a smoothed model 
* get a subsampled model
* get a depth slice by z-depth coordinate (interpolated slice)
* get a volume slice of the model from a BoundingBox (interpolated volume)
* write a .vtr file of the sliced volume so that it can beviewed in 3D

### Step 0

Load packages

In [ ]:
#load all packages
from sys import argv
import numpy as np
import pickle
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize
from gnam.model.gridmod3d import gridmod3d as gm
from gnam.model.bbox import bbox as bb
import copy

### Step 1

Read and decompress NAM ndarray model

In [ ]:
# Put in the Fully Qualified Name of the file
ifqn = './model_data/raw_nam_2017_vp_vs_rho_Q_model_dz10_depth6000.npz'

#decompress
data = np.load(ifqn)
props = data['props'] #4D ndarray of subsurface model


# Get header data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('Header Data:')
print(' x:',xdata)
print(' y:',ydata)
print(' z:',zdata)
print()


# individual parameters from the headers 
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx #notice that this can be computed

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy #notice that this can be computed

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz #notice that this can be computed


print('Individual Header Data Parameters:')
print(' xmin,dx,nx,xmax = %d,%d,%d,%d' %(xmin,dx,nx,xmax))
print(' ymin,dy,ny,ymax = %d,%d,%d,%d' %(ymin,dy,ny,ymax))
print(' zmin,dz,nz,zmax = %d,%d,%d,%d' %(zmin,dz,nz,zmax))

### Step 2

Instantiate the GriddelModel3D object

In [ ]:
nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d.shape:',gm3d.shape)

### Step 3

To QC, get a depth slice and plot it

In [ ]:
depth = 100 # This is an INDEX! So, know your header data and its mapping to the model!
iprop = 0

# index order: p      x  y  z
surf    = gm3d[iprop, :, :, depth].T
print('surf.shape',surf.shape)

# Get min max to normalize surface
vp_min = np.min(surf)
vp_max = np.max(surf)
print('vp_min:',vp_min)
print('vp_max:',vp_max)
surf_norm = Normalize(vp_min,vp_max)

# compute coordinates
xc  = dx*np.arange(nx) + xmin
yc  = dy*np.arange(ny) + ymin
xyc = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
sc = ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()


### Step 4

Now we will slice the model. Afterwards, we will plot another depth slice to QC. Reminder: one must understand their header data to be able to do this correctly. The smoothing is done with a Gaussian smoothing function which takes sigmas in number of SAMPLES.  For exmaple, if $dz=10m$ and $dx=dy=50m$ then the sigma for $z$ should be 5X more samples than it is for $x$ and $y$.

NOTE: This step will take some time.  Maybe 0.5 - 2 hours depending on the machine.

In [ ]:
# set sigmas
zsmp = 5 # 1=50m sigma, 2=100 sigma, 3=150m sigma, etc...
z_sig = zsmp*(50/dz) # tested at dz=10m and was good, so assume scale by that)
y_sig = z_sig*(dz/dy)
x_sig = z_sig*(dz/dx)
sig_meters = y_sig*50
print('sigma (m):',sig_meters)

# smooth
gm3d.smoothXYZ(x_sig,y_sig,z_sig)

### Step 5

Prepare to compress and pickle smoothed GriddedModel3D: make sure the path and file name are good and varify x,y,x "header" data.

In [ ]:
# get ndarray and header data
smth_props = gm3d.getNPArray()

xmin = gm3d.get_gorigin()[0]
dx   = gm3d.get_deltas()[0]
nx   = gm3d.get_npoints()[0]

ymin = gm3d.get_gorigin()[1]
dy   = gm3d.get_deltas()[1]
ny   = gm3d.get_npoints()[1]

zmin = gm3d.get_gorigin()[2]
dz   = gm3d.get_deltas()[2]
nz   = gm3d.get_npoints()[2]

# header data
xdata = np.array([xmin,dx,nx])
ydata = np.array([ymin,dy,ny])
zdata = np.array([zmin,dz,nz])

print('Header Data:')
print(' x-header:',xdata.astype(np.int32))
print(' y-header:',ydata.astype(np.int32))
print(' z-header:',zdata.astype(np.int32))
print()

# calculate maxDepth from header data
maxDepth = (-zmin) + (nz-1)*dz 

# fqn
out_dir = './model_data/'
fname   = 'smoothed_full_nam_2017_vp_vs_rho_Q_model_dz' 
fname  += str(int(dz)) + '_depth' + str(int(maxDepth)) + '_sig' + str(int(sig_meters)) + '.npz'
ofqn    = out_dir + fname

print('Output FQN:\n',ofqn)


### Step 6

Compress and store the numpy array of the SMOOTHED NAM model

In [ ]:
np.savez_compressed(ofqn,props=smth_props,xd=xdata,yd=ydata,zd=zdata)
print(ofqn)

### Step 7

Decompress the ndarray of the smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# copy from the line above, or put in the Fully Qualified Name of the file
ifqn = './model_data/smoothed_full_nam_2017_vp_vs_rho_Q_model_dz10_depth6000_sig250.npz'

#decompress
data = np.load(ifqn)
props = data['props'] #4D ndarray of subsurface model


#header/meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('Header Data:')
print(' x:',xdata)
print(' y:',ydata)
print(' z:',zdata)
print()


# individual parameters from the headers 
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx #notice that this can be computed

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy #notice that this can be computed

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz #notice that this can be computed


print('Individual Header Data Parameters:')
print(' xmin,dx,nx,xmax = %d,%d,%d,%d' %(xmin,dx,nx,xmax))
print(' ymin,dy,ny,ymax = %d,%d,%d,%d' %(ymin,dy,ny,ymax))
print(' zmin,dz,nz,zmax = %d,%d,%d,%d' %(zmin,dz,nz,zmax))


# instantiate GriddedModel3D object
nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d:',gm3d)
print('gm3d.shape:',gm3d.shape)


#free up some memory if needed
#del props 

### Step 8

Get another depth slice and plot it to QC smoothing and the compress/decompress

In [ ]:
depth = 100 # This is an INDEX! So, know your header data and its mapping to the model!
iprop = 0

# index order: p      x  y  z
surf    = gm3d[iprop, :, :, depth].T
print('surf.shape',surf.shape)

# Get min max to normalize surface
vp_min = np.min(surf)
vp_max = np.max(surf)
print('vp_min:',vp_min)
print('vp_max:',vp_max)
surf_norm = Normalize(vp_min,vp_max)

# compute coordinates
xc  = dx*np.arange(nx) + xmin
yc  = dy*np.arange(ny) + ymin
xyc = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
sc = ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Step 9

Now we are going to subsample model. One may want to do this if they can use a courser grid, which reduced the memory footprint and it will reduce computational cost associated with simultions that use the model. 

Here we are going to subsample the model such that $dz=dx=dy=100m$.  Again, PAY ATTENTION to your HEADER data. The arguments are is[xyz] the first SAMPLE, and id[xyz] is the subsampling interval in SAMPLES.

NOTE: We will start the subsampled grid at 5 zamples in $z$. Why? In the future we will use this model to  generate a mesh. And, each cell will be 100 meters. We will use ever point as a proxy for an entire cell in the mesh. For $x$ and $y$ this is fine because the mesh will be half a cell larger on both ends, but for $z$ it may not make sense to have the mess begin above ground (negitve $z$).

In [ ]:
sub_dxy = 2  # every second sample = 100m
sub_dz  = 10 # every tenth sample = 100m
sub_isz = 5  # start at the 5th sample -> at 50m -> that the cell in a mesh starts at z=0m 
gm3d.subsample(isz=sub_isz,idz=sub_dz,idx=sub_dxy,idy=sub_dxy) # idx=idy=2 by default
print('gm3d:',gm3d)

### Step 10

Again, we will compress and pickled the now subsampled and smoothed model. So here we prepare to compress and pickle and make sure the path and file name are good and varify x,y,x "header" data.

In [ ]:
# get ndarray and header data
subsmp_props = gm3d.getNPArray()

xmin = gm3d.get_gorigin()[0]
dx   = gm3d.get_deltas()[0]
nx   = gm3d.get_npoints()[0]

ymin = gm3d.get_gorigin()[1]
dy   = gm3d.get_deltas()[1]
ny   = gm3d.get_npoints()[1]

zmin = gm3d.get_gorigin()[2]
dz   = gm3d.get_deltas()[2]
nz   = gm3d.get_npoints()[2]

# header data
xdata = np.array([xmin,dx,nx])
ydata = np.array([ymin,dy,ny])
zdata = np.array([zmin,dz,nz])

print('Header Data:')
print(' x-header:',xdata.astype(np.int32))
print(' y-header:',ydata.astype(np.int32))
print(' z-header:',zdata.astype(np.int32))
print()

# calculate maxDepth from header data
maxDepth = nz*dz # NOTICE: this is different from how it was calculated previously.

# fqn
out_dir = './model_data/'
fname   = 'subsmp_smth_full_nam_2017_vp_vs_rho_Q_model_dz' 
fname  += str(int(dz)) + '_depth' + str(int(maxDepth)) + '_sig' + str(int(sig_meters)) + '.npz'
ofqn    = out_dir + fname

print('Output FQN:\n',ofqn)


### Step 11

Compress and store the numpy array of the SUBSAMPLED, SMOOTHED NAM model

In [ ]:
np.savez_compressed(ofqn,props=subsmp_props,xd=xdata,yd=ydata,zd=zdata)
print(ofqn)

### Step 12 

Decompress the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# copy from the line above, or put in the Fully Qualified Name of the file
ifqn = './model_data/subsmp_smth_full_nam_2017_vp_vs_rho_Q_model_dz100_depth6000_sig250.npz'

#decompress
data = np.load(ifqn)
props = data['props'] #4D ndarray of subsurface model


#header/meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('Header Data:')
print(' x:',xdata)
print(' y:',ydata)
print(' z:',zdata)
print()


# individual parameters from the headers 
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx #notice that this can be computed

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy #notice that this can be computed

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz #notice that this can be computed


print('Individual Header Data Parameters:')
print(' xmin,dx,nx,xmax = %d,%d,%d,%d' %(xmin,dx,nx,xmax))
print(' ymin,dy,ny,ymax = %d,%d,%d,%d' %(ymin,dy,ny,ymax))
print(' zmin,dz,nz,zmax = %d,%d,%d,%d' %(zmin,dz,nz,zmax))


# instantiate GriddedModel3D object
nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d:',gm3d)
print('gm3d.shape:',gm3d.shape)


#free up some memory if needed
#del props 

### Step 13

Get another depth slice and plot it to QC subsampling of the smoothed model as well as the compress/decompress.

HOWEVER, this time we will get an interpolated slice. We need to change the depth index that we use because the model has been subsampled. By indexing alone, we can not get the exact same depth slice because we have started the subsampled model at $z=50m$, so we will use a FLOAT_INDEX so that we can get the sampe depth of 1000m used above.

In [ ]:
depth = 1000.0 # Note I am using a FLOAT_INDEX -> a depth of 1000m 
iprop = 0

# Get interpolated depth slice
surf = gm3d.depthValsSliceFromZFloat(depth,iprop)
print('surf.shape',surf.shape)

# Get min max to normalize surface
vp_min = np.min(surf)
vp_max = np.max(surf)
print('vp_min:',vp_min)
print('vp_max:',vp_max)
surf_norm = Normalize(vp_min,vp_max)

# compute coordinates
xc  = dx*np.arange(nx) + xmin
yc  = dy*np.arange(ny) + ymin
xyc = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
sc = ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf,cmap=plt.cm.jet,norm=surf_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Step 14

Here we will create a BoundingBoxy (just a 4-sided, 2D polygon) which we can use to slice a volume were the $x$ and $y$ extents are contained within the polygon.  The ressulting GriddedModel3D will have $x'$ and $y'$ coordinates which are perpindicular to each other, but oblique to the initial $x$ and $y$.

The BoundingBox can be rotated and translated. And will be ploted on top of the previous depth slice. The solid-line box is the inital bbox, and the dotted line is the rotated and translated bbox.

In [ ]:
# Calculate corners for bbox
# Top left corner
tlc_y = (2/3)*dy*ny + ymin
tlc_x = (1/3)*dx*nx + xmin

# Top right corner
trc_y = tlc_y
trc_x = (2/3)*dx*nx + xmin

# Lower right corner
lrc_y = (1/3)*dy*ny + ymin
lrc_x = trc_x

# Lower left corner
llc_y = lrc_y
llc_x = tlc_x

# Create a point loop structure (surface loop) from corner points
'''
c_loop = np.array([[tlc_x,tlc_y],[trc_x,trc_y],
                   [lrc_x,lrc_y],[llc_x,llc_y],
                   [tlc_x,tlc_y]])
''';
c_loop = np.array([[llc_x,llc_y],[tlc_x,tlc_y],
                   [trc_x,trc_y],[lrc_x,lrc_y],
                   [llc_x,llc_y]])

# Instantiate bbox from surface/corner loop
mybbox = bb(c_loop)


# Make a copy of the bbox and then rotate the copy
rot_bbox = copy.deepcopy(mybbox)
rot_deg = 45
rot_bbox.rotate(rot_deg)

# Translate rotated bbox
# coordinates:       x     y
#tran_x = -3500
#tran_y = -11500
tran_x = 10500
tran_y = -5500
rot_bbox.translate(tran_x,tran_y)


# Plot both bboxes
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf,cmap=plt.cm.jet,norm=surf_norm,zorder=0)
ax.plot(mybbox.getCLoop()[:,0],mybbox.getCLoop()[:,1],c='black',zorder=3)
ax.plot(rot_bbox.getCLoop()[:,0],rot_bbox.getCLoop()[:,1],c='black',linestyle='dotted',zorder=3)
ax.set_title('NAM Model w/ Bbox')
plt.show()

### Step 15

Compress and pickle the bounding box.

In [ ]:
ofqn  = './model_data/rot_bbox_deg' + str(int(rot_deg)) 
ofqn += '_tranX' + str(int(tran_x)) + '_tranY' + str(int(tran_y)) + '.npz'
print('Output FQN:',ofqn)
print()

# Compress and pickle
print('bbox before pickle:\n',rot_bbox)
f = open(ofqn, 'wb')
pickle.dump(rot_bbox, f)
f.close()
print()


### Step 16

Decompress bounding box for QC'ing

In [ ]:
# Decompress and pickle
ifqn = './model_data/rot_bbox_deg45_tranX10500_tranY-5500.npz' # copy from output above
f = open(ifqn, 'rb')
dill_bbox = pickle.load(f) # 'Dill' get it? 
f.close()
print('bbox after pickle :\n',dill_bbox)

### Step 17

Using the rotated and translated bbox created in the previous cell, we will get a new interpolated volume slice. 

In [ ]:
vslice_gm3d = gm3d.slice_volume_by_bbox(dill_bbox)
print(vslice_gm3d)

### Step 18

Yet again, we will compress and pickled the now volume-sliced, subsampled, and smoothed model. So here we prepare to compress and pickle and make sure the path and file name are good and varify x,y,x "header" data.

In [ ]:
# get ndarray and header data
vslice_props = vslice_gm3d.getNPArray()

xmin = vslice_gm3d.get_gorigin()[0]
dx   = vslice_gm3d.get_deltas()[0]
nx   = vslice_gm3d.get_npoints()[0]

ymin = vslice_gm3d.get_gorigin()[1]
dy   = vslice_gm3d.get_deltas()[1]
ny   = vslice_gm3d.get_npoints()[1]

zmin = vslice_gm3d.get_gorigin()[2]
dz   = vslice_gm3d.get_deltas()[2]
nz   = vslice_gm3d.get_npoints()[2]

# header data
xdata = np.array([xmin,dx,nx])
ydata = np.array([ymin,dy,ny])
zdata = np.array([zmin,dz,nz])

print('Header Data:')
print(' x-header:',xdata.astype(np.int32))
print(' y-header:',ydata.astype(np.int32))
print(' z-header:',zdata.astype(np.int32))
print()

# calculate maxDepth from header data
maxDepth = nz*dz # NOTICE: this is different from how it was calculated previously.

# fqn
out_dir = './model_data/'
fname   = 'vsliced_subsmp_smth_full_nam_2017_vp_vs_rho_Q_model_dz' 
fname  += str(int(dz)) + '_depth' + str(int(maxDepth)) + '_sig' + str(int(sig_meters)) + '.npz'
ofqn    = out_dir + fname

print('Output FQN:\n',ofqn)


### Step 19

Compress and store the numpy array of the VOLUME-SLICED, SUBSAMPLED, SMOOTHED NAM model

In [ ]:
np.savez_compressed(ofqn,props=vslice_props,xd=xdata,yd=ydata,zd=zdata)
print(ofqn)

### Step 20 

Decompress the ndarray of the subsampled, smoothed NAM model and instantiate a new GriddedModel3D object for QC'ing

In [ ]:
# copy from the line above, or put in the Fully Qualified Name of the file
ifqn = './model_data/vsliced_subsmp_smth_full_nam_2017_vp_vs_rho_Q_model_dz100_depth6000_sig250.npz'

#decompress
data = np.load(ifqn)
props = data['props'] #4D ndarray of subsurface model


#header/meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('Header Data:')
print(' x:',xdata)
print(' y:',ydata)
print(' z:',zdata)
print()


# individual parameters from the headers 
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx #notice that this can be computed

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy #notice that this can be computed

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz #notice that this can be computed


print('Individual Header Data Parameters:')
print(' xmin,dx,nx,xmax = %d,%d,%d,%d' %(xmin,dx,nx,xmax))
print(' ymin,dy,ny,ymax = %d,%d,%d,%d' %(ymin,dy,ny,ymax))
print(' zmin,dz,nz,zmax = %d,%d,%d,%d' %(zmin,dz,nz,zmax))


# instantiate GriddedModel3D object
nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
dill_gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('vslice_gm3d:',vslice_gm3d)


#free up some memory if needed
#del props 

### Step 21

Overlay a depth slice of the volume-sliced model on top of the full model.  Besure to zoom in on the corners and on the edges to QC that the sliced volume is within the bounding box and that the interpolated values make sense.

NOTE: We will make use of the "notebook" windowing-backend so that we can ZOOM!

In [ ]:
# we want to be able to zoom into the corners and edges
%matplotlib notebook  

depth = 9 # This is an INDEX! shoudl be at 1050m in depth
iprop = 0

# index order:          p      x  y  z
vslice_surf = dill_gm3d[iprop, :, :, depth].T
full_surf   = gm3d[iprop, :, :, depth].T
print('vslice_surf.shape',vslice_surf.shape)
print('full_surf.shape',full_surf.shape)


# Get min max to normalize surface
vp_min = np.min(vslice_surf)
vp_max = np.max(vslice_surf)
print('vp_min:',vp_min)
print('vp_max:',vp_max)
full_surf_norm = Normalize(vp_min,vp_max)


# Get coordinates
vslice_xyc = vslice_gm3d.getGlobalCoordsPointsXY()
full_xyc   = gm3d.getGlobalCoordsPointsXY()


#Plot
fig, ax = plt.subplots(1,figsize=(8,8))

# Plot the fullslice
sc = ax.scatter(full_xyc[:,0],full_xyc[:,1],s=1,c=full_surf.flatten(),cmap=plt.cm.jet,norm=full_surf_norm,zorder=0)

# Overlay plot of the bbox filled with white
#ax.fill(dill_bbox.getCLoop()[:,0],dill_bbox.getCLoop()[:,1],c='white',zorder=1)

# Overlay plot of volume depth slice
ax.scatter(vslice_xyc[:,0],vslice_xyc[:,1],s=1,c=vslice_surf.flatten(),cmap=plt.cm.jet,norm=full_surf_norm,zorder=2)

# Over lay dotted plot of bbox surface
ax.plot(dill_bbox.getCLoop()[:,0],dill_bbox.getCLoop()[:,1],c='black',linestyle='dotted',zorder=4)

# Set other plot options
ax.set_title('Full NAM Model with Slice Overlay')
fig.colorbar(sc)
plt.show()


### Step -1

Now that we have a model we want, we can use Paraview to view both it and the full model if we convert them to VTK/VTR files.  So, lets do it...

NOTE: Now that you know how to slice a volume out, take a look at the notebook where the Groningen events and station data are fetched, then find plot a bbox and fine-tune its shape and location for the desired coverage. Then come back here and create a volume slice.

In [ ]:
from gnam.vtkutils.write import write_vtk_gridded_model_3d


#vk_props = dill_gm3d.getNPArray().transpose(0,3,2,1)
vk_props = dill_gm3d.getNPArray()
print('vk_props.shape:',vk_props.shape)
vk_xdata = np.zeros((3))
vk_ydata = np.zeros((3))
vk_zdata = np.zeros((3))
vk_xdata[1] = dill_gm3d.get_deltas()[0]
vk_ydata[1] = dill_gm3d.get_deltas()[1]
vk_zdata[1] = dill_gm3d.get_deltas()[2]
vk_xdata[2] = vk_props.shape[1]
vk_ydata[2] = vk_props.shape[2]
vk_zdata[2] = vk_props.shape[3]
print('vk_xdata:',vk_xdata)
print('vk_ydata:',vk_ydata)
print('vk_zdata:',vk_zdata)

# Copy and past ifqn from previous volume-slice decompression here, but do so without the extension
vtk_ofqn = './model_data/vsliced_subsmp_smth_full_nam_2017_vp_vs_rho_Q_model_dz100_depth6000_sig250'
print('vtk_ofqn:',vtk_ofqn+'.vtr')
write_vtk_gridded_model_3d(vtk_ofqn,vk_props,vk_xdata,vk_ydata,vk_zdata)

print()

full_vk_props = gm3d.getNPArray()
print('full_vk_props.shape:',full_vk_props.shape)
full_vk_xdata = np.zeros((3))
full_vk_ydata = np.zeros((3))
full_vk_zdata = np.zeros((3))
full_vk_xdata[1] = gm3d.get_deltas()[0]
full_vk_ydata[1] = gm3d.get_deltas()[1]
full_vk_zdata[1] = gm3d.get_deltas()[2]
full_vk_xdata[2] = full_vk_props.shape[1]
full_vk_ydata[2] = full_vk_props.shape[2]
full_vk_zdata[2] = full_vk_props.shape[3]
print('full_vk_xdata:',full_vk_xdata)
print('full_vk_ydata:',full_vk_ydata)
print('full_vk_zdata:',full_vk_zdata)

# Copy and past ifqn from previous full decompression here, but do so without the extension
full_vtk_ofqn = './model_data/subsmp_smth_full_nam_2017_vp_vs_rho_Q_model_dz100_depth6000_sig250'
print('full_vtk_ofqn:',full_vtk_ofqn+'.vtr')
write_vtk_gridded_model_3d(full_vtk_ofqn,full_vk_props,full_vk_xdata,full_vk_ydata,full_vk_zdata)

print('\nNow open the files in Parview and enjoy!')